<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=764f481daabb3a854ceac86baac02f36043614d18ec97cfd52e704782f536349
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-19 09:24:22
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 5.89 K (0.04%)
Current PnL: -16.69 L (-11.23%)
CY Booked + Current PnL: -5.25 L (-3.53%)
-------------------
Total profit:  2.10 L
Total loss:  -18.79 L
-------------------
Total Booked + Current PnL: 21.73 L (17.85%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.21 L (56.83%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,104.27,57.0,M-SC,3.69,88032.0,-12745.0,12835.0,-0.24,-12.65,14.58,0.09,245.0,-0.99,0.64,15.65,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,12.69,61.0,H-MC,3.46,194490.0,29820.0,13517.0,-0.82,18.11,6.95,26.31,89.0,2.21,1.42,27.54,X40N,NTT,AC
83,VOLTAS,1530.00,2.28,62.0,H-MC,2.71,214725.0,22983.0,14773.0,0.37,11.99,6.88,19.69,99.0,1.56,1.57,19.60,XY25,NTT,AC
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.94,112016.0,11512.0,17598.0,0.02,11.45,15.71,28.96,79.0,0.65,0.82,49.31,MH,ATH,METALS
56,RELIANCE,1533.00,-12.16,57.0,H-LC,4.96,220007.0,9641.0,19141.0,-0.33,4.58,8.70,13.68,37.0,0.50,1.61,21.86,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,0.62,60.0,H-MC,1.93,249444.0,1713.0,73337.0,0.29,0.69,29.40,30.30,92.0,0.02,1.82,14.07,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-31.08,50.0,H-MC,4.10,199665.0,-1155.0,69923.0,-0.31,-0.58,35.02,34.25,80.0,-0.02,1.46,4.59,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-13.71,43.0,H-LC,7.73,217712.0,-34056.0,87259.0,-0.19,-13.53,40.08,21.13,27.0,-0.39,1.59,17.11,X40,ATH,PAINTS
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302603.0,-13239.0,125913.0,0.16,-4.19,41.61,35.67,5.0,-0.11,2.21,9.08,X40N,ATH,FMCG
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200955.0,817.0,20075.0,-0.15,0.41,9.99,10.44,4.0,0.04,1.47,5.99,X40,NTT,FMCG
30,HCLTECH,1943.91,1.94,53.0,H-LC,9.69,233176.0,-8748.0,72028.0,-0.58,-3.62,30.89,26.15,8.0,-0.12,1.71,10.19,X40,ATH,IT
50,NESTLEIND,1377.00,-9.11,53.0,H-LC,4.37,279770.0,14344.0,42441.0,-1.12,5.40,15.17,21.40,11.0,0.34,2.05,12.88,XY25,NTT,FMCG


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,-0.29,68.0,H-LC,1.47,247388.0,1237.0,77977.0,-0.49,0.50,31.52,32.18,16.0,0.02,1.81,36.73,X200,ATH,IT
41,ITC,452.0,-37.92,51.0,H-LC,2.37,200955.0,817.0,20075.0,-0.15,0.41,9.99,10.44,4.0,0.04,1.47,5.99,X40,NTT,FMCG
50,NESTLEIND,1377.0,-9.11,53.0,H-LC,4.37,279770.0,14344.0,42441.0,-1.12,5.40,15.17,21.40,11.0,0.34,2.05,12.88,XY25,NTT,FMCG
56,RELIANCE,1533.0,-12.16,57.0,H-LC,4.96,220007.0,9641.0,19141.0,-0.33,4.58,8.70,13.68,37.0,0.50,1.61,21.86,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.44,54.0,H-LC,5.74,215117.0,10617.0,30805.0,-0.51,5.19,14.32,20.25,10.0,0.34,1.57,14.38,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7155.56,69.0,L-SC,5.25,79604.0,-14206.0,90828.0,2.63,-15.14,114.10,81.67,269.0,-0.16,0.58,54.16,XR,NTT,CERAMICS
70,TANLA,1963.11,-18.28,73.0,H-SC,12.21,219524.0,-51237.0,381181.0,2.54,-18.92,173.64,121.86,133.0,-0.13,1.61,71.73,AR,ATH,IT
26,GREENPANEL,537.00,235.27,62.0,M-SC,1.78,149694.0,-33384.0,112360.0,2.03,-18.23,75.06,43.14,230.0,-0.30,1.09,40.81,XY24,NTT,MISC
21,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.33,85824.0,-27725.0,67775.0,1.68,-24.42,78.97,35.27,268.0,-0.41,0.63,110.06,XR,NTT,HOTELS
8,AWL,485.00,-64.66,56.0,H-MC,4.04,251690.0,-50061.0,211495.0,1.52,-16.59,84.03,53.50,116.0,-0.24,1.84,11.79,XY24,NTT,FMCG


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VALIANTORG,838.0,-323.33,52.0,H-SC,6.25,124287.0,-39318.0,160629.0,-2.40,-24.03,129.24,74.15,139.0,-0.24,0.91,59.32,XR,NTT,CHEMICALS
53,RAJESHEXPO,518.0,1857.07,61.0,L-SC,2.66,53411.0,-83766.0,83861.0,-1.66,-61.06,157.01,0.07,267.0,-1.00,0.39,32.83,OX40N,NTT,JEWELLERY
80,VAIBHAVGBL,521.0,63.64,68.0,H-SC,6.62,141852.0,-40923.0,151994.0,-1.26,-22.39,107.15,60.77,125.0,-0.27,1.04,27.05,XR,NTT,JEWELLERY
50,NESTLEIND,1377.0,-9.11,53.0,H-LC,4.37,279770.0,14344.0,42441.0,-1.12,5.40,15.17,21.40,11.0,0.34,2.05,12.88,XY25,NTT,FMCG
16,CAMPUS,393.0,-23.39,62.0,M-SC,0.79,159661.0,-8114.0,63960.0,-0.95,-4.84,40.06,33.28,210.0,-0.13,1.17,23.75,XY24,NTT,FOOTWEAR


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,104.27,57.0,M-SC,3.69,88032.0,-12745.0,12835.0,-0.24,-12.65,14.58,0.09,245.0,-0.99,0.64,15.65,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,53.0,H-SC,9.41,122850.0,-19629.0,39152.0,-0.02,-13.78,31.87,13.70,163.0,-0.50,0.90,45.00,OX40N,NTT,BANKS
44,KANSAINER,340.0,-66.69,67.0,H-SC,1.23,230058.0,-39609.0,75942.0,0.94,-14.69,33.01,13.47,138.0,-0.52,1.68,16.36,XY24,NTT,PAINTS
18,CERA,9475.0,-20.05,47.0,H-SC,2.09,145970.0,-29933.0,71949.0,-0.02,-17.02,49.29,23.89,149.0,-0.42,1.07,26.06,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.51,54.0,H-MC,7.44,108784.0,-23051.0,65270.0,-0.14,-17.48,60.00,32.03,98.0,-0.35,0.80,21.85,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.47,56.0,H-LC,12.30,296429.0,-49527.0,124382.0,-0.73,-14.32,41.96,21.64,1.0,-0.40,2.17,5.01,X40,ATH,IT
39,INFY,2275.00,-16.01,57.0,H-LC,8.16,327168.0,14280.0,157401.0,-0.30,4.56,48.11,54.87,3.0,0.09,2.39,11.52,X40,BTT,IT
41,ITC,452.00,-37.92,51.0,H-LC,2.37,200955.0,817.0,20075.0,-0.15,0.41,9.99,10.44,4.0,0.04,1.47,5.99,X40,NTT,FMCG
82,VBL,671.64,-15.98,43.0,H-LC,6.56,302603.0,-13239.0,125913.0,0.16,-4.19,41.61,35.67,5.0,-0.11,2.21,9.08,X40N,ATH,FMCG
1,ABB,7934.00,-37.43,69.0,H-LC,9.84,261566.0,-53.0,119274.0,0.24,-0.02,45.60,45.57,7.0,-0.00,1.91,11.30,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,45.95,67.0,M-MC,7.46,230231.0,5269.0,162382.0,0.39,2.34,70.53,74.53,192.0,0.03,1.68,35.35,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.69,61.0,H-MC,3.46,194490.0,29820.0,13517.0,-0.82,18.11,6.95,26.31,89.0,2.21,1.42,27.54,X40N,NTT,AC
83,VOLTAS,1530.00,2.28,62.0,H-MC,2.71,214725.0,22983.0,14773.0,0.37,11.99,6.88,19.69,99.0,1.56,1.57,19.60,XY25,NTT,AC
17,CAMS,5211.76,0.82,67.0,H-SC,3.45,114596.0,12592.0,31331.0,0.43,12.34,27.34,43.06,122.0,0.40,0.84,31.57,X40N,ATH,MISC
29,HAVELLS,2069.16,0.62,60.0,H-MC,1.93,249444.0,1713.0,73337.0,0.29,0.69,29.40,30.30,92.0,0.02,1.82,14.07,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,244.55,-43.66,65.0,H-MC,7.94,112016.0,11512.0,17598.0,0.02,11.45,15.71,28.96,79.0,0.65,0.82,49.31,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.92,49.0,M-SC,0.64,99832.0,8334.0,70421.0,-0.32,9.11,70.54,86.07,223.0,0.12,0.73,44.97,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.40,39.0,H-SC,11.51,132903.0,9567.0,124198.0,-0.61,7.76,93.45,108.45,119.0,0.08,0.97,31.84,AR,ATH,MISC
85,WIPRO,420.00,-9.26,60.0,M-LC,5.65,159235.0,8290.0,101162.0,-0.04,5.49,63.53,72.51,53.0,0.08,1.16,11.85,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,65.0,H-MC,6.30,184099.0,4291.0,108121.0,0.58,2.39,58.73,62.52,88.0,0.04,1.35,37.29,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7155.56,69.0,L-SC,5.25,79604.0,-14206.0,90828.0,2.63,-15.14,114.10,81.67,269.0,-0.16,0.58,54.16,XR,NTT,CERAMICS
60,SAMMAANCAP,326.00,-172.73,64.0,M-SC,3.22,83298.0,-18912.0,112302.0,-0.26,-18.50,134.82,91.37,208.0,-0.17,0.61,35.22,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.33,85824.0,-27725.0,67775.0,1.68,-24.42,78.97,35.27,268.0,-0.41,0.63,110.06,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.00,48.0,H-SC,12.50,89290.0,-11673.0,106202.0,0.06,-11.56,118.94,93.63,148.0,-0.11,0.65,32.99,SR,ATH,CHEMICALS
48,MASFIN,398.61,-17.34,51.0,H-SC,12.98,94155.0,-3825.0,25431.0,-0.08,-3.90,27.01,22.05,152.0,-0.15,0.69,36.86,XR,ATH,FINANCE


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SHALBY,327.00,1108.93,70.0,M-SC,4.60,164504.0,-16639.0,62100.0,0.88,-9.19,37.75,25.10,235.0,-0.27,1.20,31.83,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3878.38,63.0,M-SC,5.34,124077.0,-23291.0,75104.0,-0.45,-15.80,60.53,35.16,236.0,-0.31,0.91,26.73,XY24,NTT,AUTO
74,TITAGARH,1548.00,1.18,69.0,H-SC,3.16,165970.0,-26045.0,104926.0,0.20,-13.56,63.22,41.08,158.0,-0.25,1.21,40.64,XY24,NTT,ENGINEERING
70,TANLA,1963.11,-18.28,73.0,H-SC,12.21,219524.0,-51237.0,381181.0,2.54,-18.92,173.64,121.86,133.0,-0.13,1.61,71.73,AR,ATH,IT
21,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.33,85824.0,-27725.0,67775.0,1.68,-24.42,78.97,35.27,268.0,-0.41,0.63,110.06,XR,NTT,HOTELS


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.10
1,25,43.97
2,50,74.50


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.56
LC    32.10
MC    23.33
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.37
X40      15.39
X40N     11.94
XR       11.80
AR        8.78
XY25      8.25
OX40N     8.21
X200      1.81
MH        1.70
X5K       1.45
SR        1.29
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.31
H-LC    25.39
H-MC    20.34
M-SC    13.65
M-LC     5.65
M-MC     2.68
L-SC     1.60
L-LC     1.06
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.68,-12.49,73.82
FMCG,11.92,-3.90,40.25
FINANCE,9.73,-15.55,60.81
MISC,7.47,-12.40,74.89
BANKS,6.35,-15.21,75.81
PAINTS,5.87,-14.87,32.00
ELECTRICAL,5.78,-4.14,61.53
AC,3.71,4.66,11.31
INSURANCE,3.45,-6.35,42.83


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2825528.0,22
XR,1299022.0,14
AR,1296073.0,9
X40,744167.0,10
X40N,572294.0,9
OX40N,524468.0,10
XY25,336798.0,6
SR,244454.0,2
X5K,119207.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3405205.0,29
M-SC,1379043.0,17
H-LC,1219778.0,14
H-MC,1086678.0,15
M-LC,377802.0,4
M-MC,310799.0,2
L-SC,242464.0,3
L-MC,60484.0,1
L-LC,38723.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1021052.0      6
M-SC       XY24       794743.0      7
H-SC       AR         783961.0      5
           XR         755280.0      7
H-MC       XY24       484058.0      4
H-LC       X40        461145.0      5
           AR         350959.0      2
M-MC       XY24       310799.0      2
H-SC       X40N       286535.0      4
           OX40N      250533.0      4
           SR         244454.0      2
H-MC       X40        212353.0      4
H-LC       X40N       210672.0      3
M-SC       AR         161153.0      2
L-SC       XR         158603.0      2
M-LC       XY24       157433.0      2
M-SC       OX40N      124804.0      4
H-MC       XY25       124191.0      2
M-LC       X5K        119207.0      1
M-SC       XR         115372.0      2
           XY25       112302.0      1
H-MC       XR         108121.0      1
M-LC       XR         101162.0      1
L-SC       OX40N       83861.0      1
H-LC       X200        77977.0      1
H-MC       X40N        75087.0      2
M-SC       X40         70669.0      1
H-MC       OX40N       65270.0      1
H-SC       MH          63390.0      1
H-LC       XY25        61582.0      2
L-MC       XR          60484.0      1
H-LC       XY24        57443.0      1
L-LC       XY25        38723.0      1
H-MC       MH          17598.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
